In [1]:
import sys
import numpy as np
import pandas as pd
import cv2
import pickle

from os import listdir 
from os.path import isfile, join
from tifffile import imread, imsave
from glob import glob

np.random.seed(42)


import logging
logger = logging.getLogger()  # Logger

logger.setLevel("INFO")
#logging.info("Hello")

In [2]:
def write_pickle(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
        
def read_pickle(path):
    with open(path, "rb") as f:
        obj = pickle.load(f)
    return obj

In [208]:
first = read_pickle(path="/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing_lbl_tile/512x512/tilecells_tenth.pkl")

In [209]:
print(first)

      tiles  Pseudonym          celltypes
0     146.0  IQ5WIKH4I         Beta cells
1     199.0  IQ5WIKH4I        myCAF_POSTN
2     169.0  IQ5WIKH4I            B cells
3     183.0  IQ5WIKH4I            T cells
4     140.0  IQ5WIKH4I     Macrophages_M2
...     ...        ...                ...
8023  198.0  IQ5WIKH4I        myCAF_ACTA2
8024  185.0  IQ5WIKH4I  Endothelial cells
8025  131.0  IQ5WIKH4I       Ductal cells
8026   72.0  IQ5WIKH4I        myCAF_ACTA2
8027  227.0  IQ5WIKH4I     Classical_KRT7

[8028 rows x 3 columns]


In [210]:
img = first["tiles"].to_list()
img_lst = []

for i in img:
    r = int(i)
    t = str(r)
    img_lst.append(t)

In [211]:
first["tiles"] = img_lst

In [212]:
first["img"] = first["Pseudonym"] + "_" + first["tiles"]

In [185]:
#first["label"] = np.where((first["celltypes"] == 'Basal') | (first["celltypes"].str.contains('Classical')), "cancer", "non-cancer")

In [213]:
first["label"] = np.where((first["celltypes"] == 'Basal'), 'Basal',
                           np.where((first["celltypes"] == 'Classical_KRT7'), 'Classical_KRT7',
                                     np.where((first["celltypes"] == 'Classical_CEACAM'), 'Classical_CEACAM',
                                               np.where((first["celltypes"] == 'Classical_TFF1'), 'Classical_TFF1',
                                                         np.where((first["celltypes"] == 'Classical_REG4'), 'Classical_REG4',
                                                                 'non-cancer')))))    

In [ ]:
label_dict = {"Basal": "Basal", "Classical_KRT7": "Classical_KRT7", "Classical_CEACAM": "Classical_CEACAM", "Classical_TFF1": "Classical_TFF1", "Classical_REG4": "Classical_REG4"}
first["label"] = first.apply(lambda x: label_dict.get(x["celltypes"], "non-cancer"), axis=1)

In [214]:
print(first)

     tiles  Pseudonym          celltypes            img           label
0      146  IQ5WIKH4I         Beta cells  IQ5WIKH4I_146      non-cancer
1      199  IQ5WIKH4I        myCAF_POSTN  IQ5WIKH4I_199      non-cancer
2      169  IQ5WIKH4I            B cells  IQ5WIKH4I_169      non-cancer
3      183  IQ5WIKH4I            T cells  IQ5WIKH4I_183      non-cancer
4      140  IQ5WIKH4I     Macrophages_M2  IQ5WIKH4I_140      non-cancer
...    ...        ...                ...            ...             ...
8023   198  IQ5WIKH4I        myCAF_ACTA2  IQ5WIKH4I_198      non-cancer
8024   185  IQ5WIKH4I  Endothelial cells  IQ5WIKH4I_185      non-cancer
8025   131  IQ5WIKH4I       Ductal cells  IQ5WIKH4I_131      non-cancer
8026    72  IQ5WIKH4I        myCAF_ACTA2   IQ5WIKH4I_72      non-cancer
8027   227  IQ5WIKH4I     Classical_KRT7  IQ5WIKH4I_227  Classical_KRT7

[8028 rows x 5 columns]


In [215]:
tiles = first["img"].to_list()
res = {}
for tile in tiles:
    t = first[first["img"]==tile]
    r = t["label"].to_list()
    r = max(set(r), key=r.count)
    #print(t)
    #print(r)
    res[tile] = r
    #break

In [216]:
write_pickle(path='/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing_lbl_tile/512x512/lbs_tenth_6class.pkl', obj = res)